In [1]:
from datasets import FirstTokenRepeatedOnceImmediately
import pandas as pd
import torch
import numpy as np

/home/kevin/miniconda3/envs/jax-hf/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data = FirstTokenRepeatedOnceImmediately(num_points=1000)

In [3]:
# 0th index is data, 1xt index is label
data.train_data.dataset.tensors[0][:10], data.train_data.dataset.tensors[1][:10]

(tensor([[ 3,  3,  6, 10, 13, 19, 14, 15,  8, 12],
         [ 4,  4, 14, 11, 18,  9, 10,  1,  3,  6],
         [ 8,  8, 16,  6, 13, 15, 18, 17,  9,  5],
         [18, 16, 19,  3,  4,  1,  5, 14, 10, 12],
         [16, 17,  4,  1, 11,  8, 14,  2,  6, 10],
         [ 5, 11, 12,  6,  2, 16, 19,  8,  4, 14],
         [ 5,  4,  7, 14, 18, 10,  2, 13, 19,  8],
         [ 3,  3, 12, 18,  8, 10, 15, 11, 16, 17],
         [11, 11,  9, 13, 17,  5, 15,  7, 18,  1],
         [14,  9,  8,  7, 13, 15,  3,  5,  1, 11]]),
 tensor([1, 1, 1, 0, 0, 0, 0, 1, 1, 0]))

In [4]:
def view_class_balance(dataset):
    return pd.Series(next(iter(torch.utils.data.DataLoader(dataset, batch_size=len(data.train_data))))[1]).value_counts()

print("Train data:\n", view_class_balance(data.train_data))
print("Val data:\n", view_class_balance(data.val_data))
print("Test data:\n", view_class_balance(data.test_data))

Train data:
 0    352
1    348
dtype: int64
Val data:
 1    54
0    46
dtype: int64
Test data:
 0    102
1     98
dtype: int64


In [5]:
%env XLA_PYTHON_CLIENT_PREALLOCATE=false
from transformers import BertConfig, BfBertForSequenceClassification
config = BertConfig.from_pretrained(pretrained_model_name_or_path="../../brunoflow/models/bert/config-toy.json")
m = BfBertForSequenceClassification(config=config)


env: XLA_PYTHON_CLIENT_PREALLOCATE=false


2023-03-24 01:43:48.525731: E external/org_tensorflow/tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:267] failed call to cuInit: CUDA_ERROR_UNKNOWN: unknown error
/home/kevin/code/rycolab/transformers/src/transformers/models/bert/modeling_bf_bert.py:181: UserWarning: Explicitly requested dtype <class 'jax.numpy.int64'> requested in zeros is not available, and will be truncated to dtype int32. To enable more dtypes, set the jax_enable_x64 configuration option or the JAX_ENABLE_X64 shell environment variable. See https://github.com/google/jax#current-gotchas for more.
  bf.Parameter(jnp.zeros(self.position_ids.shape, dtype=jnp.int64), name="position_ids"),


In [6]:
np.log(2)

0.6931471805599453

In [7]:
import brunoflow as bf
from jax import numpy as jnp 
x, y = next(iter(torch.utils.data.DataLoader(data.train_data, batch_size=128)))
# x
outputs = m(bf.Node(jnp.array(x.numpy())), labels=bf.Node(jnp.array(y.numpy())))
outputs.loss, outputs.logits

(node(name: /, val: 0.6930131912231445, grad: 0.0),
 node(name: +, val: [[-1.37126527e-03 -9.01138628e-05]
  [-1.80962647e-03  1.66876882e-04]
  [ 5.31584024e-04 -3.31441406e-04]
  [ 7.98313471e-04 -7.68861282e-05]
  [-1.86409964e-03  1.02115337e-04]
  [-1.75975915e-03  6.41789084e-05]
  [-7.61804986e-04 -1.17064876e-04]
  [-1.88836711e-03  2.01237723e-04]
  [ 5.31887345e-04 -3.31450050e-04]
  [-1.77783147e-03  2.18887672e-05]
  [-5.76508581e-04  2.60645320e-04]
  [-1.75975636e-03  6.41542501e-05]
  [-9.21344035e-04  3.09743162e-04]
  [-1.88837666e-03  2.01235773e-04]
  [-1.37104711e-03 -9.01628882e-05]
  [-1.44251692e-03 -9.23413172e-05]
  [-1.93395978e-03  6.85869381e-05]
  [-1.37112266e-03 -9.01401872e-05]
  [-8.70748248e-04  1.07544358e-04]
  [ 4.08811873e-04 -2.61874229e-04]
  [-1.48952263e-03  8.65522743e-05]
  [-1.82982709e-03  1.83797645e-04]
  [-9.21550323e-04  3.09744762e-04]
  [-1.77767815e-03  2.18122805e-05]
  [-1.90212997e-03  8.95837729e-05]
  [-1.88837503e-03  2.0122296